In [33]:
class  RawDataUserFilter:

    def __init__(self,textfileloc,numfile,klmfile,pathtofile):
        self.text = textfileloc
        self.num = numfile
        self.klm = klmfile
        self.path = pathtofile
    def textmaker(self):
        #opens and readies a text file containing all filenames
        filenames = open(self.text, 'r')
        filestring = filenames.read()
        listOfFilenames = filestring.split('\n')
        if self.num == 'all':
            listOfFilenames = listOfFilenames
        elif type(self.num)==int:
            listOfFilenames = listOfFilenames[0:self.num]
        else:
            print("error a non command value was inserted for numfile")
            listOfFilenames = {}
        return listOfFilenames

    
    def klmconverter(self):
        klmNames = (open(self.klm,'r')).read()
        klmNameList = klmNames.split('\n')
        nameList = []
        for item in klmNameList:
            nameList.append(int(float(item)))
        return nameList

    def getReplyKlmBawBB(self):
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        RepToBawList = []
        KLMframe = []
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            for entry in readable.iterrows():
                try:
                    alp = int(entry[1]['in_reply_to_user_id'])
                except:
                    continue
                if alp in KLMlist:
                    KLMframe.append(entry)
        
        framelist =[]
        for item in KLMframe:
            framelist.append(item[1])
        RepToKlmList = pandas.DataFrame(framelist)
        RepToBawList.append(readable[readable['in_reply_to_user_id'] == 18332190])
        return(RepToKlmList,RepToBawList)

    def getReplyKlmBaw(self):
        #initialises the list of KLM tweets and files
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        #two empty lists for storage purposes
        RepToBawList = []
        KLMframe = []
        #reads through every file one at a time
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            #Checks if the tweet is sent to one of the accounts of KLM
            for entry in readable.iterrows():
                try:
                    alp = int(entry[1]['in_reply_to_user_id'])
                except:
                    continue
                if alp in KLMlist:
                    KLMframe.append(entry)
        #creates a dataframe of tweets sent to klm
        framelist =[]
        for item in KLMframe:
            framelist.append(item[1])
        RepToKlmList = pandas.DataFrame(framelist)
        #gets a list of dataframes for tweets set to british airways
        RepToBawList.append(readable[readable['in_reply_to_user_id'] == 18332190])
        return(RepToKlmList,RepToBawList)
    
    def getKlmAll(self):
        #makes two empty list to dump all the dataframes into one for KLM one for british airways
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        frameklm = []
        framebrit = []
        KLMframe = []
        #creates a for-loop which sifts through all the files
        for item in listOfFilenames:
            #reminder to see where the system is currently computing
            print("handling dataset:"+item)

            #takes the filename and inserts it into the Json to Dataframe object to turn it into a dataframe
            readable = jsonRead(self.path+item).JSONtoDf()

            #initialises a counter
            usernum=0

            #checks for each user if the value is a dict, if not it gives a corresponding error message
            for i in list(readable['user']):
                if type(i)!= dict:
                    if type(i)==float:
                        print("nan value changed to empty dict")
                    elif type(i)!=float:
                        print('weird value changed to empty dict')
                    #changes non-dict values to be empty
                    readable['user'][usernum] = {}
                usernum+=1

                #takes the user dicts and turns them into rows of a dataframe
            subframe = pandas.DataFrame(list(readable['user']))
            subframe['ident'] = list(readable['id'])
            #a dataframe containing all tweets whose user id is equal to the id for a certain json
            for entry,index in zip(subframe['id'],subframe['ident']):
                try:
                    if int(entry) in KLMlist:
                        KLMframe.append(index)
                except:
                    print('fault')
                    continue
            BritWaylist = list(subframe[subframe['id']==18332190].index)

            #appends all the dataframes per json to the corresponding empty list
            frameklm.append(readable[readable['id'].isin(KLMframe)])
            framebrit.append(readable[readable.index.isin(BritWaylist)])

            #returns the lists of datasets
        return(frameklm,framebrit) 
    
    def getKlmAllBB(self):
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        frameklm = []
        framebrit = []
        KLMframe = []
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            usernum=0
            for i in list(readable['user']):
                if type(i)!= dict:
                    readable['user'][usernum] = {}
                usernum+=1
            subframe = pandas.DataFrame(list(readable['user']))
            subframe['ident'] = list(readable['id'])
            for entry,index in zip(subframe['id'],subframe['ident']):
                try:
                    if int(entry) in KLMlist:
                        KLMframe.append(index)     
                except:
                    continue
            BritWaylist = list(subframe[subframe['id']==18332190].index)
            frameklm.append(readable[readable['id'].isin(KLMframe)])
            framebrit.append(readable[readable.index.isin(BritWaylist)])
        return(frameklm,framebrit) 

In [6]:
class FrameConcat:
    
    def __init__(self,doubleframe):
        self.frames = doubleframe
    
    def concatinator(self):
        concatframes = []
        for i in range(2):
            Rep= self.frames[i]
            try:
                concatframes.append(pandas.concat(Rep,ignore_index=True))
            except:
                concatframes.append(Rep)
        return concatframes


In [36]:
#necessairy libraries
import pandas
import json

#class object
class jsonRead:
        #sets the file location as the self.name
    def __init__(self , filename):
        self.name = filename
        
        #function which reads the json lines and puts them into a dataframe
    def JSONtoDf(self):
        #list to dump json files into
        data = []
        #initialises linecounter for later usage
        linecount = 0
        # takes the self.name opens it and assigns it to the variable f
        with open(self.name) as f:
            #line is one line of json
            for line in f:
                linecount +=1
                #tries to append the json line to the list, if it fails it prints the json that caused the exception
                try:
                    data.append(json.loads(line))
                except:
                    print("fault in line:"+str(linecount))
        #turns the list into a dataframe and returns it
        son = pandas.DataFrame(data)
        return son


In [34]:
a =RawDataUserFilter('json/filenames.txt',10,'KLMnumb shortlist.txt', 'json/').getReplyKlmBawBB()
b = FrameConcat(a).concatinator()
b[0]

fault in line:3318
fault in line:4785
fault in line:304
fault in line:5331
fault in line:2935


,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,retweet_count,retweeted,retweeted_status,scopes,source,text,timestamp_ms,truncated,user,withheld_in_countries
32,NaN,None,Mon May 30 09:58:33 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 737221522150948866, 'id_str'...",0.0,False,low,None,7.372217e+17,...,0.0,False,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@KLM that's just the front of your line https:...,1464602313611,False,"{'id': 1128067884, 'id_str': '1128067884', 'na...",NaN
38,NaN,None,Mon May 30 09:59:02 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372218e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","@KLM bedankt, DM is onderweg",1464602342959,False,"{'id': 147178542, 'id_str': '147178542', 'name...",NaN
56,NaN,None,Mon May 30 09:59:55 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 737221888095719424, 'id_str'...",0.0,False,low,None,7.372220e+17,...,0.0,False,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@KLM here's the back of it...I can't even see ...,1464602395566,False,"{'id': 1128067884, 'id_str': '1128067884', 'na...",NaN
71,NaN,None,Mon May 30 10:00:27 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372221e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",@KLM my flight is the 15.25 from Schipol to Lo...,1464602427763,False,"{'id': 339927723, 'id_str': '339927723', 'name...",NaN
78,NaN,None,Mon May 30 10:00:38 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372222e+17,...,0.0,False,NaN,NaN,"<a href=""http://www.twitter.com"" rel=""nofollow...",@KLM Toch jammer dat jullie united niet gaan v...,1464602438062,False,"{'id': 2316714537, 'id_str': '2316714537', 'na...",NaN
82,NaN,None,Mon May 30 10:00:46 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372222e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@KLM Ik heb nog niks geboekt. Ik wilde dit eer...,1464602446286,False,"{'id': 1120249927, 'id_str': '1120249927', 'na...",NaN
116,NaN,None,Mon May 30 10:02:40 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372227e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@KLM will the 9:30am flight from Newcastle to ...,1464602560410,False,"{'id': 3300448673, 'id_str': '3300448673', 'na...",NaN
124,NaN,None,Mon May 30 10:03:03 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372228e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@KLM @The_West_Guy,1464602583218,False,"{'id': 1168751665, 'id_str': '1168751665', 'na...",NaN
141,NaN,None,Mon May 30 10:03:46 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372230e+17,...,0.0,False,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",@KLM Rebooked by phone for flight tomorrow. Pr...,1464602626522,False,"{'id': 48942020, 'id_str': '48942020', 'name':...",NaN
142,NaN,None,Mon May 30 10:03:48 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,False,low,None,7.372230e+17,...,0.0,False,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@KLM can you confirm that flights are taking o...,1464602628691,False,"{'id': 239195655, 'id_str': '239195655', 'name...",NaN


In [35]:
b[1]

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id_str,retweet_count,retweeted,retweeted_status,scopes,source,text,timestamp_ms,truncated,user
0,None,None,Fri Jun 03 23:48:24 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,None,738880040688099328,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@British_Airways BEST AIRLINE IN THE WORLD,1464997704084,False,"{'id': 2742509213, 'id_str': '2742509213', 'na..."
1,None,None,Fri Jun 03 23:48:58 +0000 2016,"{'hashtags': [{'text': 'whatloyalty', 'indices...",NaN,0,False,low,None,738880184946991104,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways according to ITV you treat Go...,1464997738478,False,"{'id': 350787916, 'id_str': '350787916', 'name..."
2,None,None,Fri Jun 03 23:51:31 +0000 2016,"{'hashtags': [{'text': 'BAIOW16', 'indices': [...",NaN,0,False,low,None,738880827027775488,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways @IsleOfWightFest I want to wi...,1464997891562,False,"{'id': 2663207107, 'id_str': '2663207107', 'na..."
3,None,None,Fri Jun 03 23:55:01 +0000 2016,"{'hashtags': [{'text': 'BAIOW16', 'indices': [...",NaN,0,False,low,None,738881706925658112,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways @IsleOfWightFest I want to wi...,1464998101346,False,"{'id': 50715108, 'id_str': '50715108', 'name':..."
4,None,None,Fri Jun 03 23:55:02 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,None,738881712206323714,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@British_Airways kids renamed BA as ""Baddest A...",1464998102605,False,"{'id': 53986750, 'id_str': '53986750', 'name':..."
5,None,None,Fri Jun 03 23:57:35 +0000 2016,"{'hashtags': [{'text': 'BAIOW16', 'indices': [...",NaN,0,False,low,None,738882355130171393,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways @IsleOfWightFest I want to wi...,1464998255890,False,"{'id': 524795355, 'id_str': '524795355', 'name..."
6,None,None,Fri Jun 03 23:58:17 +0000 2016,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,None,738882527776100352,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways lost all of the above with Al...,1464998297052,False,"{'id': 2471119426, 'id_str': '2471119426', 'na..."
7,None,None,Fri Jun 03 23:59:37 +0000 2016,"{'hashtags': [{'text': 'BAIOW16', 'indices': [...",NaN,0,False,low,None,738882864041824256,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways @IsleOfWightFest I want to wi...,1464998377224,False,"{'id': 1907252137, 'id_str': '1907252137', 'na..."
8,None,None,Sat Jun 04 00:02:01 +0000 2016,"{'hashtags': [{'text': 'BA202', 'indices': [70...",NaN,0,False,low,None,738883467778379776,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways: When I checked in last night...,1464998521166,False,"{'id': 634821500, 'id_str': '634821500', 'name..."
9,None,None,Sat Jun 04 00:15:55 +0000 2016,"{'hashtags': [{'text': 'BAIOW16', 'indices': [...",NaN,0,False,low,None,738886968415784965,...,NaN,0,False,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@British_Airways @IsleOfWightFest I want to wi...,1464999355783,False,"{'id': 3572458276, 'id_str': '3572458276', 'na..."
